In [ ]:
import pandas as pd
import h5py
from pathlib import Path
import logging
import logging.config
import yaml
import h5py
import numpy as np
import matplotlib.pyplot as plt

import storage
import extraction

%matplotlib inline

In [ ]:
def setupLogging():
    with open('../config/logConfig.yml', 'rt') as file:
        config = yaml.safe_load(file.read())
        logging.config.dictConfig(config)

In [ ]:
setupLogging()

In [ ]:
# Read TSV files

train_df = pd.read_table('../data/Train_GCC-training.tsv', header = None, names = ['caption', 'url'] )
#validate_df = pd.read_table('../data/Validation_GCC 1.1.0-Validation.tsv', header = None, names = ['caption', 'url']) 

train_df.head(20)

In [ ]:
# Get last stored index
start_index = storage.get_last_stored_index("train")
print(start_index)

In [ ]:
# Extracts image and caption from tsv and store padded(image),caption and HTTPStatus code
# into a train or validation hdf5 file

# Set image size
size = 300 , 300 

# Extraction and Storage
#extraction.request_data_and_store(validate_df, size, "validate", start_index) 
#extraction.request_data_and_store(train_df, size, "train", start_index - 100)
extraction.request_data_and_store(train_df, size, "train", start_index)
 

In [ ]:
df = storage.get_statuses("validation")

In [ ]:
print(df["statuscode"].unique().tolist())
print(df["statuscode"].value_counts().tolist())

plt.hist(df["statuscode"], bins = 18)
plt.style.use('ggplot')
plt.show()

print(df["statuscode"].value_counts())

In [ ]:

labels = df["statuscode"].value_counts().keys().tolist()
sizes = df["statuscode"].value_counts().tolist()
explode = np.zeros(18)
explode[1:18] = 0.1

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, explode=explode, radius=4000, shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
from itertools import islice
df = pd.DataFrame(columns = ['statuscode'])
df.astype({'statuscode': 'int32'})
df["statuscode"] = ['a','b','c','d','e','f','g','h','i','j','k','l']

In [ ]:
for index, row in islice(df["statuscode"].iteritems(), 5, None):
    print(str(index) +"--"+ row)